In [1]:
import numpy as np
import matplotlib.pyplot as plt

import supervisely_lib as sly

from tqdm import tqdm
from collections import defaultdict

%matplotlib inline

In [2]:
address = 'http://192.168.1.69:5555'
token = 'YGPDnuBkhFmcQ7VNzSEjhgavjg4eFR4Eq1C3jIY4HgV3SQq2JgkXCNtgZy1Fu2ftd4IKui8DsjrdtXjB853cMtBevpSJqFDYiaG1A5qphlH6fFiYYmcVZ5fMR8dDrt5l'
team_name = 'dima'
workspace_name = 'work'

src_project_name = 'roads_inf'

In [3]:
api = sly.Api(address, token)

team_id = api.team.get_info_by_name(team_name)['id']
workspace_id = api.workspace.get_info_by_name(workspace_name, team_id)['id']

src_project_id = api.project.get_info_by_name(src_project_name, workspace_id)['id']

src_meta_json = api.project.get_meta(src_project_id)
src_meta = sly.ProjectMeta.from_json(src_meta_json)

In [4]:
classes_mapping = {
    'road': 'road_unet'
}
iou_threshold = 0.5

In [5]:
def iou(mask_1, mask_2):
    intersection = (mask_1 * mask_2).sum()
    union = mask_1.sum() + mask_2.sum() - intersection
    if union == 0:
        return 0.0
    return intersection / union


def compute_overlaps(masks1, masks2):
    overlaps = np.zeros((masks1.shape[0], masks2.shape[0]))
    for i in range(overlaps.shape[1]):
        mask2 = masks2[i]
        for j in range(overlaps.shape[0]):
            mask1 = masks1[j]
            overlaps[j, i] = iou(mask1, mask2)
    return overlaps


def compute_matches(gt_masks, pred_masks, iou_threshold):
    overlaps = compute_overlaps(pred_masks, gt_masks)
    pred_match = -1 * np.ones([pred_masks.shape[0]])
    gt_match = -1 * np.ones([gt_masks.shape[0]])
    for i in range(len(pred_masks)):
        sorted_ixs = np.argsort(overlaps[i])[::-1]
        for j in sorted_ixs:
            if gt_match[j] > 0:
                continue
            iou = overlaps[i, j]
            if iou < iou_threshold:
                break
            gt_match[j] = i
            pred_match[i] = j
            break

    return gt_match, pred_match


def compute_precision_recall(gt_masks, pred_masks, iou_threshold=0.5):
    if len(gt_masks) == 0:
        return 0, len(pred_masks), 0
    elif len(pred_masks) == 0:
        return 0, 0, len(gt_masks)
    gt_masks = np.stack(gt_masks, axis=0)
    pred_masks = np.stack(pred_masks, axis=0)
    gt_match, pred_match = compute_matches(gt_masks, pred_masks, iou_threshold)

    tp = np.sum(pred_match > -1)
    all_pred = len(pred_match)

    all_gt = len(gt_match)
    return tp, all_pred, all_gt


def process(ann, metric_res):
    img_size = ann.img_size
    for cls_gt, cls_pred in classes_mapping.items():
        masks_gt, masks_pred = [], []
        for label in ann.labels:
            if label.obj_class.name == cls_gt:
                mask = np.zeros(img_size, np.uint8)
                label.geometry.draw(mask, 1)
                masks_gt.append(mask)
            if label.obj_class.name == cls_pred:
                mask = np.zeros(img_size, np.uint8)
                label.geometry.draw(mask, 1)
                masks_pred.append(mask)
                
        tp, all_pred, all_gt = compute_precision_recall(masks_gt, masks_pred, iou_threshold)
        pair_name = cls_gt + ':' + cls_pred
        metric_res[pair_name]['true-positive'] += tp
        metric_res[pair_name]['all-ground-truth'] += all_gt
        metric_res[pair_name]['all-predictions'] += all_pred

In [7]:
metric_results = defaultdict(lambda: {'true-positive': 0, 'all-ground-truth': 0, 'all-predictions': 0})
for dataset_info in api.dataset.get_list(src_project_id):
    src_dataset_id = dataset_info['id']
    src_dataset_name = dataset_info['name']

    print('Project/Dataset: {}/{}'.format(src_project_name, src_dataset_name))
    
    for image_info in tqdm(api.image.get_list(src_dataset_id)):
        src_image_ext = image_info['meta']['mime'].split('/')[1]

        ann_json = api.annotation.download(src_dataset_id, image_info['id'])
        ann = sly.Annotation.from_json(ann_json, src_meta)
        process(ann, metric_results)

 30%|███       | 3/10 [00:00<00:00, 26.42it/s]

Project/Dataset: roads_inf/ds1


100%|██████████| 10/10 [00:00<00:00, 27.66it/s]


In [15]:
all_prec = []
all_rec = []
for cls_pair in metric_results:
    met_vals = metric_results[cls_pair]
    precision = met_vals['true-positive'] / met_vals['all-predictions'] if met_vals['all-predictions'] > 0 else 0
    recall = met_vals['true-positive'] / met_vals['all-ground-truth'] if met_vals['all-ground-truth'] > 0 else 0
    all_prec.append(precision)
    all_rec.append(recall)
    print('Results for classes {!r} and {!r}.'.format(*cls_pair.split(':')))
    print('Precision: {}'.format(precision))
    print('Recall: {}'.format(recall))
    print()
print('Average for all classes.')
print('Precision: {}'.format(np.mean(all_prec)))
print('Recall: {}'.format(np.mean(all_rec)))

Results for classes 'road' and 'road_unet'.
Precision: 0.8888888888888888
Recall: 0.8

Average for all classes.
Precision: 0.8888888888888888
Recall: 0.8


In [14]:
f = lambda: {"a": 1}

In [15]:
d = defaultdict(f)

In [16]:
d['a']['a'] += 1

In [17]:
d

defaultdict(<function __main__.<lambda>()>, {'a': {'a': 2}})